In [ ]:
import pandas as pd
import pingouin as pg
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.reporting import get_clusters_table
from nilearn.plotting import plot_glass_brain
from nilearn.image import new_img_like
import nibabel as nib
from glob import glob
import numpy as np
from pathlib import Path
import sys
sys.path.append('..')
from config.paths import DATA_DIR
from plotting.plot import pretty_behav_plot, plot_by_nvoxels

%load_ext autoreload
%autoreload 2

In [ ]:
data_dir = Path(DATA_DIR) / 'experiment_1'

## Behavioral data

In [ ]:
behav_df = []

all_subjdirs = data_dir.glob('sub-*')
for i, s in enumerate(all_subjdirs):
     this_subj = pd.read_csv(list(s.glob('func/*_beh.tsv'))[0], sep='\t')
     this_subj['Subject'] = f'sub-{i+1:03d}'
     behav_df.append(this_subj)
behav_df = pd.concat(behav_df)
avg_behav = behav_df.groupby(['Subject', 'Consistent']).mean().reset_index()

In [ ]:
pg.ttest(avg_behav[avg_behav['Consistent']==1]['Hit'], 
         avg_behav[avg_behav['Consistent']==0]['Hit'], paired=True)

In [ ]:
pretty_behav_plot(avgdata=avg_behav)

## MVPA

In [ ]:
all_mvpa_files = data_dir.glob('mvpa_results/exp1_*.csv')
mvpa_df = []
for f in all_mvpa_files:
    mvpa_df.append(pd.read_csv(f))
mvpa_df = pd.concat(mvpa_df)

In [ ]:
mvpa_avg = mvpa_df.groupby(['roi', 'subject_id', 'n_voxels', 'hemisphere', 'congruency']).mean(numeric_only=True).reset_index()

evc_results = mvpa_avg[mvpa_avg['roi']=='ba-17-18']
lvc_results = mvpa_avg[mvpa_avg['roi']=='ba-19-37']

In [ ]:
# check that decoding is overall above chance in EVC
pg.ttest(evc_results.groupby(['subject_id']).mean(numeric_only=True)['classifier_info'], 0.0)

In [ ]:
# and in LVC
pg.ttest(lvc_results.groupby(['subject_id']).mean(numeric_only=True)['classifier_info'], 0.0)

In [ ]:
pg.ttest(evc_results.groupby('subject_id').mean(numeric_only=True)['classifier_info'],
         lvc_results.groupby('subject_id').mean(numeric_only=True)['classifier_info'],
         paired=True)

Main comparison: congruent vs. incongruent averaged across voxel sizes

In [ ]:
pg.ttest(evc_results[evc_results['congruency']=='congruent'].groupby('subject_id').mean(numeric_only=True)['classifier_info'],
         evc_results[evc_results['congruency']=='incongruent'].groupby('subject_id').mean(numeric_only=True)['classifier_info'],
         paired=True)

In [ ]:
mvpa_avg

ANOVA to check for interactions between ROI and congruency

In [ ]:
mvpa_avg.groupby(['subject_id', 'roi', 'congruency']).mean(numeric_only=True).reset_index()

In [ ]:
aov = pg.rm_anova(data=mvpa_avg.groupby(['subject_id', 'roi', 'congruency']).mean(numeric_only=True).reset_index(), 
                  dv='classifier_info', within=['roi', 'congruency'], subject='subject_id')
aov.round(4)

### Early Visual Cortex

In [ ]:
_ = plot_by_nvoxels(evc_results, n_perms=10000, right_part=True)

### Late Visual Cortex

In [ ]:
_ = plot_by_nvoxels(lvc_results, n_perms=10000, right_part=True)

## Univariate

In [ ]:
# Get paths to contrast files

all_contr_files = []

contr_dir = data_dir / 'derivatives' / 'spm-preproc' / 'derivatives' / 'spm-stats' / 'contrasts'
for s in sorted(contr_dir.glob('sub-*')):
    thiscontr_path = s / 'test' / 'exp1_cong_incong'
    all_contr_files.append(list(thiscontr_path.glob('con*.nii'))[0])

In [ ]:
design_matrix = pd.DataFrame([1] * len(all_contr_files), columns=['intercept'])
second_level_model = SecondLevelModel().fit(
    all_contr_files, design_matrix=design_matrix
)

In [ ]:
z_map = second_level_model.compute_contrast(second_level_stat_type='t', output_type="z_score")

In [ ]:
thresholded_map, threshold = threshold_stats_img(
    z_map,
    alpha=0.001,
    height_control="fpr",
    cluster_threshold=10,
    two_sided=True,
)
thresholded_map = new_img_like(thresholded_map, np.nan_to_num(thresholded_map.get_fdata()))

In [ ]:
get_clusters_table(z_map, threshold, cluster_threshold=10,
                   two_sided=True)

In [ ]:
disp = plot_glass_brain(z_map, threshold=threshold, draw_cross=False, colorbar=True)

## Information-activation coupling

In [ ]:
all_cong_maps = []
all_incong_maps = []

infocoupl_dir = data_dir / 'infocoupl_maps'

for m in sorted(infocoupl_dir.glob('*.nii.gz')):
    thismapfile = str(m)
    if 'incongruent' not in m.name and 'congruent' in m.name:
        all_cong_maps.append(thismapfile)
    elif 'incongruent' in m.name:
        all_incong_maps.append(thismapfile)

In [ ]:
n_subjects = len(all_cong_maps)
subject_list = [f'sub-{i:03d}' for i in range(1, n_subjects + 1)]
condition_effect = np.hstack(([1] * n_subjects, [-1] * n_subjects))
subject_effect = np.vstack((np.eye(n_subjects), np.eye(n_subjects)))
paired_design_matrix = pd.DataFrame(
    np.hstack((condition_effect[:, np.newaxis], subject_effect)),
    columns=["congruent vs incongruent"] + subject_list,
)

In [ ]:
second_level_model_paired = SecondLevelModel().fit(
    all_cong_maps + all_incong_maps, design_matrix=paired_design_matrix
)

In [ ]:
stat_maps_paired = second_level_model_paired.compute_contrast(
    "congruent vs incongruent", output_type="all"
)

In [ ]:
thresholded_map, threshold = threshold_stats_img(
    stat_maps_paired["z_score"],
    alpha=0.001,
    height_control="fpr",
    cluster_threshold=10,
    two_sided=False,
)

In [ ]:
disp=plot_glass_brain(
    thresholded_map,
    threshold=threshold,
    colorbar=True,
    plot_abs=False
)

In [ ]:
get_clusters_table(stat_maps_paired['z_score'], threshold,
                   cluster_threshold=10, two_sided=False)